In [ ]:
import pandas as pd 
# 웹 서버에 요청을 보내고 응답 메시지 받는 기능능
import requests
# 크롤링 기능
from bs4 import BeautifulSoup as bs
# 웹 브라우져를 제어 
from selenium import webdriver
# 브라우져의 태그를 검색
from selenium.webdriver.common.by import By
# 키보드의 특수 이벤트(ENTER, DELETE, CLEAR, ...)
from selenium.webdriver.common.keys import Keys

- 네이버 증권 
    - 종목코드를 이용하여 검색 
        - url의 규칙 확인
    - 뉴스 탭에 더보기 버튼을 선택
        - 클릭 이벤트 발생하여 페이지 이동
        - url의 규칙 확인
    - 뉴스 리스트에 있는 하이퍼링크의 주소를 모두 리스트의 형태로 저장
        - requests라이브러리를 이용하여 해당 주소로 요청 
            - 정보를 확인
            - 기사의 내용이 모두 응답 메시지에 존재한다면 제목, 본문 내용으로 1차원 리스트 생성 
            - 모든 기사 내용을 리스트 추가(2차원 리스트)
            - pandas 이용하여 데이터프레임을 생성
            - csv 파일로 저장 (파일의 이름에 현재 시간을 대입해서 파일 중복 방지)

In [ ]:
url = "https://finance.naver.com/item/main.naver?code="

code = '012450'

In [ ]:
# 웹 브라우져 오픈
driver = webdriver.Chrome()

In [ ]:
# 특정 주소를 요청을 보낸다 . 
driver.get(url+code)

In [ ]:
# driver에서 html 코드를 불러와서(웹 브라우져에 page_source) 
# BeautifulSoup를 이용하여 데이터를 파싱(class 생성)
soup = bs(driver.page_source, 'html.parser')

In [ ]:
len(driver.page_source)

In [ ]:
# div 태그 중에 class의 값이 news_section 인 태그의 개수를 확인
len(
    soup.find_all(
        'div', 
        attrs = {
            'class' : 'news_section'
        }
    )
)
# div태그에서 _replaceNewsLink 태그는 iframe 태그에 존재
# 다른 화면(다른 url)에 태그가 존재 

In [ ]:
# div 태그 중 class가 news_section이라는 태그를 추출 
div_data = soup.find(
    'div', 
    attrs = {
        'class' : 'news_section'
    }
)

In [ ]:
# div_data에서 li태그를 모두 검색
li_list = div_data.find_all('li')

In [ ]:
len(li_list)

In [ ]:
# li_list 각 원소들의 첫번째 a태그를 선택하여 
# href의 속성 값을 이용하여 새로운 리스트 생성
href_list = []

for li_data in li_list:
    # li_data에서 첫번째 a태그를 추출 -> href 속성의 값을 추출
    data = li_data.find('a')['href']
    # href_list에 추가 
    href_list.append(data)

In [ ]:
href_list

In [ ]:
base_url = "https://finance.naver.com"

In [ ]:
href_list[0]

In [ ]:
res = requests.get(base_url+href_list[0])

In [ ]:
print(res.text)

In [ ]:
# selenium을 이용해서 base_url + href_list[0]에 요청 
driver.get(base_url + href_list[2])

In [ ]:
# html문서를 bs을 이용하여 데이터 파싱
news_soup = bs(driver.page_source, 'html.parser')

In [ ]:
# h2태그 중 id가 'title_area'인 태그를 선택하고 텍스트 추출
news_title = news_soup.find(
    'h2', 
    attrs={
        'id' : 'title_area'
    }
).get_text()
news_title

In [ ]:
# div 태그중에 id 가 'newsct_article'인 태그를 선택하고 텍스트 추출 
news_content = news_soup.find(
    'div', 
    attrs = {
        'id' : 'newsct_article'
    }
).get_text().replace('\n', '')

news_content

In [ ]:
news_data = []
news_data.append(news_title)
news_data.append(news_content)

In [ ]:
news_data

In [ ]:
news_dict = {}
news_dict['title'] = news_title
news_dict['content'] = news_content
news_dict

In [50]:
news_dict2 = {
    'title' : news_title, 
    'content' : news_content
}
news_dict2

{'title': '한화, 수소연료전지 글로벌 인증 획득…“친환경 기술 경쟁력 확보”',
 'content': '한화에어로스페이스, 세계적 권위 ‘DNV’ 인증액침냉각 ESS’에 이어 ‘무탄소 동력원’ 확대[이데일리 김성진 기자] 한화에어로스페이스가 세계 최고 수준의‘200kW(킬로와트)급 선박용 수소연료전지’인증을 획득해 친환경 무탄소 선박 시장에 본격적으로 진출한다. 지난해 말 세계 최초로 개발한 액침냉각 에너지저장장치(ESS)와 함께 세계 최고 수준의 무탄소 동력원을 잇달아 확보했다..한화에어로스페이스는 12일 세계 권위의 선급 협회인 DNV로부터 AIP(개념승인) 인증을 받았다고 13일 밝혔다. AIP는 수소연료전지 등 신기술이 적용된 선박을 제작할 때 기본설계 단계부터 기술 적용의 안정성과 국제 규정 준수 여부 등을 검증해 주는 절차다.한화에어로스페이스는 지난해 KR(한국선급) AIP 인증에 이어 세계 최고 수준의 DNV 인증까지 획득하면서 글로벌 시장에서 선박용 수소연료전지의 본격적인 판매가 가능해졌다.이번에 DNV 인증을 받은 수소연료전지는 고분자 전해질 연료전지(PEMFC)다. 저온에서도 발전이 가능해 안정적이고 에너지효율이 높아 선박 등 운송용에 적합하다. 또한 병렬제어 기능으로 MW(메가와트) 단위의 용량 확장이 가능해 중대형 선박에도 적용이 가능하다.한화에어로스페이스 관계자는 “AIP의 다음 단계인 선박용 연료전지의 형식 인증도 확보해 무탄소 추진 시스템을 적용한 선박 개발 시장에서 한화오션과 함께 기술 경쟁력을 지속적으로 확보하겠다”고 밝혔다.오동조 한화에어로스페이스 연료전지개발팀장(왼쪽 세번째)과 신성호 DNV 부사장(왼쪽 네번째)이 한화에어로스페이스 판교 R＆D센터에서 열린 선박용 수소연료전지 AIP 인증 수여식에 참석했다.(사진=한화에어로스페이스.)'}

In [51]:
import time

In [52]:
# 데이터를 저장할수 있는 빈 리스트 생성 
result = []
base_url = "https://finance.naver.com"
# href_list를 이용하여 반복문을 실행
for href in href_list:
    # base_url + href_list[index]를 이용하여 webdriver에 요청
    driver.get(base_url + href)
    # 2초 정도의 딜레이 
    time.sleep(2)
    # 해당 driver에서 html를 추출하여 데이터 파싱 
    news_soup = bs(driver.page_source, 'html.parser')
    # h2태그 중 id가 title_area인 태그에서 텍스트를 추출하여 변수에 저장
    news_title = news_soup.find(
        'h2', 
        attrs = {'id' : 'title_area'}
    ).get_text()
    # div태그 중 id가 newsct_article인 태그에서 텍스트를 추철하여 변수에 저장
    news_content = news_soup.find(
        'div', 
        attrs = {'id' : 'newsct_article'}
    ).get_text().replace('\n', '')
    # news_dict 생성하여 h2태그의 텍스트와 div태그의 텍스트를 저장
    news_dict = {
        'title' : news_title, 'content' : news_content
    }
    # result에 news_dict를 추가
    result.append(news_dict)

In [53]:
result

[{'title': "방산주 연일 랠리...현대로템·한화에어로 '불기둥' [특징주]",
  'content': '현대로템 K2 전차. [파이낸셜뉴스] 국내 방산주가 장 초반 급등하고 있다. 유럽 방위비 증액 기대감이 여전한 데다 증권가에서 한화에어로스페이스에 대한 목표주가를 상향한 영향으로 풀이된다.   13일 오전 9시34분 현대로템은 전 거래일 대비 7.10% 상승한 10만1000원에 거래 중이다. 장 초반 10만3300원까지 급등하면서 사상 최고가를 경신했다. 한화에어로스페이스도 5.71% 오른 70만3000원에 거래 중이다.   유럽 방위비 증액 기대감에 연일 방산주가 상승하고 있다. 증권가에서 목표주가를 상향하면서 투자심리가 몰린 것으로 풀이된다.   장남현 한국투자증권 연구원은 현대로템에 대해 "미국의 자국 우선주의 기조가 짙어지면서 유럽 국가들이 자체 국방 강화를 위한 방위비 증액에 나서고 있다”며 “현대로템은 비유럽 지역에서 경쟁 강도가 낮아지고, 유럽 내 수요 증대에 따른 시장이 확대되는 두 가지 수혜를 모두 누릴 수 있을 것”이라고 밝혔다.   이날 KB증권은 한화에어로스페이스에 대해 목표주가를 기존 60만원에서 83만원으로 상향했다.   정동익 KB증권 연구원은 "한화에어로스페이스 주가가 올해 들어 2배 이상 급등했으나 높은 자기자본이익률(ROE)에 비해 상대적으로 주가순자산비율이 낮은 상황"이라고 말했다.\t\t'},
 {'title': '美 CPI 예상치 하회에 국내 증시도 ‘영차’… 코스피 2590대 등락 [투자360]',
  'content': '미국 증시 반등에 코스피도 ‘영차’개인·기관 ‘사자’ 외인은 순매도 중조선·방산 강세, 코스닥도 상승연합뉴스[헤럴드경제=신주희 기자] 미국 2월 소비자물가지수(CPI)가 예상치를 하회함에 따라 전날 미증시가 반등하면서 코스피가 상승 출발했다.이날 오전 9시 21분 기준 코스피 지수는 전 거래일 대비 19.81포인트(0.77%) 오른 2,594.63을 나타냈다. 지수는 전장보다 23.24포인트